In [1]:
from jax_circuits import *
from scipy.stats import unitary_group
import matplotlib.pyplot as plt

import time
import pickle

from qiskit import transpile
from qiskit.quantum_info import Operator

from topology import *

from jax import custom_jvp, jvp, grad
from jax.scipy.optimize import minimize

key = random.PRNGKey(42)

Number of ideas to try in optimization of Toffoli gates
- Try to use ancillas
- Try to find decompositions up to swaps

# Toffoli 3

In [2]:
qc = QuantumCircuit(3)
qc.ccx(0, 1, 2)
u_toff3 = Operator(qc.reverse_bits()).data

## Randomized learning

In [3]:
%%time

num_attempts = 10
key = random.PRNGKey(13)
for _ in range(num_attempts):
    key, subkey = random.split(key)
    current_placements = random_placements(3, 6, key=subkey)
    anz = Ansatz(3, 'cz', placements={'free':current_placements})
    break
    ah, lh, sh = unitary_learn_repeat(anz.unitary, 
                                      u_toff3, 
                                      anz.num_angles, 
                                      'adam', 
                                      0.1, 
                                      num_repeats=5, 
                                      target_loss=1e-4)
    if sum(sh)>0:
        print('decomposition found')

CPU times: user 376 ms, sys: 2.71 ms, total: 378 ms
Wall time: 374 ms


In [4]:
# placements = [[1, 2], [0, 1], [1, 0], [2, 0], [0, 2], [1, 2]]
anz= Ansatz(3, 'cz', placements={'free': current_placements})
anz.circuit().draw()

DAGCircuitError: "(qu)bit 1 not found in OrderedDict([(Qubit(QuantumRegister(3, 'q'), 0), <qiskit.dagcircuit.dagnode.DAGNode object at 0x7f9f0648cfa0>), (Qubit(QuantumRegister(3, 'q'), 1), <qiskit.dagcircuit.dagnode.DAGNode object at 0x7f9f0648ce80>), (Qubit(QuantumRegister(3, 'q'), 2), <qiskit.dagcircuit.dagnode.DAGNode object at 0x7f9f0648cd00>)])"

In [28]:
anz.circuit().draw()

DAGCircuitError: "(qu)bit 1 not found in OrderedDict([(Qubit(QuantumRegister(3, 'q'), 0), <qiskit.dagcircuit.dagnode.DAGNode object at 0x7fe1a7cdd7c0>), (Qubit(QuantumRegister(3, 'q'), 1), <qiskit.dagcircuit.dagnode.DAGNode object at 0x7fe1a7cdd880>), (Qubit(QuantumRegister(3, 'q'), 2), <qiskit.dagcircuit.dagnode.DAGNode object at 0x7fe1a7cdd940>)])"

In [27]:
anz.unitary(random_angles(anz.num_angles))

DeviceArray([[ 0.19073969-0.31325555j, -0.13835403+0.15000434j,
              -0.1899369 +0.3934965j , -0.14490005-0.13827404j,
               0.10087714-0.15804735j, -0.06854949-0.37335977j,
               0.22301579-0.0622921j , -0.5966718 -0.0626908j ],
             [ 0.04510305+0.2784415j , -0.01589786+0.16940373j,
              -0.40577137-0.03305881j,  0.35006016-0.48061025j,
              -0.04618294-0.05733552j, -0.42797944+0.16141689j,
              -0.3197593 -0.14336145j, -0.11609475+0.14593011j],
             [ 0.09147388-0.5086384j , -0.5384947 -0.15515459j,
               0.06733114-0.03065343j,  0.17188033-0.3292146j ,
              -0.10955666+0.13414639j, -0.0581906 +0.14316753j,
               0.25072098-0.18742867j,  0.34349164+0.07492664j],
             [-0.16379164-0.0443546j , -0.14117153+0.19880366j,
               0.05664904-0.40177065j,  0.02325146-0.06162427j,
              -0.35039145+0.11584258j,  0.03842258+0.28260976j,
               0.24127656+0.4969588j 